Setup

In [22]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent
import pandas as pd

In [23]:
# Set up the Chrome driver
options = Options()

# Tune the Driver with Argumentations
# options.add_argument("--headless")  # Run headless Chrome
# options.add_argument("--no-sandbox")
ua = UserAgent()
options.add_argument(f"user-agent={ua.random}")

Given

In [24]:
Scholars_list = ["Cass Sunstein", "Tom Tyler", "W. Kip Viscusi", "Lawrence Gostin", "Mark Lemley", "Lucian Bebchuk", "John Monahan", "Alan Auerbach", "Eric Posner", "Richard Delgado"]

In [25]:
df = pd.DataFrame(columns=['Professor Name', 'Affiliation', 'Total Number of Citations', 'h-index', 'i10-index', 'Most Cited Paper', 'Research Fields'])

In [26]:
# Global variables to store scraped data
name, affiliation, most_cited_paper, research_fields, citation_count, h_index, i10_index = None, None, None, None, None, None, None

In [27]:
# Flag to track if we’ve already switched to the author list
switched_to_author_list = False

In [28]:
driver = webdriver.Chrome(options=options)
driver.get("https://scholar.google.com/")
#driver.maximize_window()
time.sleep(random.uniform(2, 4))  # Random delay

In [ ]:
# cookies
cookies = [
    {'name': 'name', 'value': 'value'},
    {'name': 'name', 'value': 'value'},
    {'name': 'name', 'value': 'value'},
    {'name': 'name', 'value': 'value'}
]

In [ ]:
def load_cookies(driver, cookies):
    for cookie in cookies:
        driver.add_cookie(cookie)

In [ ]:
load_cookies(driver, cookies)

In [29]:
# To avoid CAPTCHA
def fake_human_behavior(driver):
    
    '''# Random mouse movements
    actions = ActionChains(driver)
    
    # Simulate moving the mouse to a random position within the viewport
    x_offset = random.randint(0, 500)  # Adjust based on your viewport size
    y_offset = random.randint(0, 500)  # Adjust based on your viewport size
    actions.move_by_offset(x_offset, y_offset).perform()
    time.sleep(random.uniform(0.5, 1.5))  # Pause after moving the mouse

    # Scroll down randomly
    scroll_amount = random.randint(200, 800)  # Adjust scroll amount
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")'''
    
    time.sleep(random.uniform(1, 2))

Functions

In [30]:
# one time use only
#search_bar = driver.find_element(By.CSS_SELECTOR, 'input#gs_hdr_tsi')

In [31]:
# Function to search for a professor
def search_professor(professor_name):
    search_bar = driver.find_element(By.CSS_SELECTOR, 'input#gs_hdr_tsi')
    search_bar.send_keys(professor_name)
    search_bar.submit()

    fake_human_behavior(driver)

In [32]:
# Function to switch to the author list (only the first time)
def switch_to_author_list():
    global switched_to_author_list
    if not switched_to_author_list:
        try:
            profile_switch = driver.find_element(By.XPATH, "//h3/a[contains(@href, '/')]")
            profile_switch.click()
            switched_to_author_list = True  # Set the flag to True, so this doesn't run again
        except Exception as e:
            print(f"Error switching to author list: {e}")

In [33]:
# Function to select the first profile
def select_first_profile():
    try:
        profile_link = driver.find_element(By.XPATH, "//h3/a[contains(@href, '/')]")
        profile_link.click()
        return True
    except Exception as e:
        print(f"Error selecting profile: {e}")
        return False
    
    fake_human_behavior(driver)

In [34]:
# Function to scrape the textual profile details (name, affiliation, most cited paper, research fields)
def scrape_text():
    global name, affiliation, most_cited_paper, research_fields
    try:
        name = driver.find_element(By.CSS_SELECTOR, "div#gsc_prf_in").text
        affiliation = driver.find_element(By.CSS_SELECTOR, "div.gsc_prf_il:nth-of-type(2)").text
        most_cited_paper = driver.find_element(By.XPATH, "//tr[@class='gsc_a_tr'][1]//a[@class='gsc_a_at']").text
        research_fields = ",\n".join([field.text for field in driver.find_elements(By.CSS_SELECTOR, "div.gsc_prf_il#gsc_prf_int a")])
    except Exception as e:
        print(f"Error scraping textual profile: {e}")
    
    fake_human_behavior(driver)

In [35]:
# Switch to the 'CITED BY' tab
def toggle_data_tab():
        data_tab_toggle = driver.find_element(By.LINK_TEXT, "CITED BY")
        data_tab_toggle.click()
        fake_human_behavior(driver)

In [36]:
# Function to switch to the 'CITED BY' tab and scrape the stats (citations, h-index, i10-index)
def scrape_stats():
    global citation_count, h_index, i10_index
    try:
        citation_count = driver.find_element(By.XPATH, "//tr[1]//td[@class='gsc_rsb_std'][1]").text
        h_index = driver.find_element(By.XPATH, "//tr[2]/td[@class='gsc_rsb_std'][1]").text
        i10_index = driver.find_element(By.XPATH, "//tr[3]/td[@class='gsc_rsb_std'][1]").text
    except Exception as e:
        print(f"Error scraping stats: {e}")
    
    fake_human_behavior(driver)

In [37]:
# Function to return to the search page after scraping
def return_to_search():
    search_btn = driver.find_element(By.CSS_SELECTOR, "a#gs_hdr_sre span.gs_ico")
    search_btn.click()
    fake_human_behavior(driver)

Scraping loop

In [38]:
for scholar in Scholars_list:
    search_professor(scholar)
    
    # Only attempt to switch to the author list the first time
    switch_to_author_list()
    
    select_first_profile()
    
    # Part 1: Scrape the textual profile details
    scrape_text()

    toggle_data_tab()
    # Part 2: Scrape the citation statistics
    scrape_stats()

    # Add the current set of data to the DataFrame using .loc[]
    df.loc[len(df)] = [name, affiliation, citation_count, h_index, i10_index, most_cited_paper, research_fields]

    return_to_search()

MoveTargetOutOfBoundsException: Message: move target out of bounds
  (Session info: chrome=129.0.6668.70)
Stacktrace:
	GetHandleVerifier [0x00007FF69919FDA5+29557]
	(No symbol) [0x00007FF699112240]
	(No symbol) [0x00007FF698FCB519]
	(No symbol) [0x00007FF6990726FF]
	(No symbol) [0x00007FF6990474FA]
	(No symbol) [0x00007FF6990688A3]
	(No symbol) [0x00007FF6990472A3]
	(No symbol) [0x00007FF6990112DF]
	(No symbol) [0x00007FF699012451]
	GetHandleVerifier [0x00007FF6994CDCBD+3363469]
	GetHandleVerifier [0x00007FF699519B47+3674391]
	GetHandleVerifier [0x00007FF69950EAEB+3629243]
	GetHandleVerifier [0x00007FF69925FC66+815670]
	(No symbol) [0x00007FF69911D6EF]
	(No symbol) [0x00007FF6991192B4]
	(No symbol) [0x00007FF699119450]
	(No symbol) [0x00007FF6991081FF]
	BaseThreadInitThunk [0x00007FF8437D257D+29]
	RtlUserThreadStart [0x00007FF844FCAF28+40]


Epilogue

In [18]:
# After the loop, quit the driver
driver.quit()

In [19]:
# Save the data to CSV
df.to_csv('professor_profiles.csv', index=False)

In [20]:
# Print "Scraping complete. Data saved to 'professor_profiles.csv'."
print("Scraping complete. Data saved to 'professor_profiles.csv'.")

Scraping complete. Data saved to 'professor_profiles.csv'.
